Stabel Diffusion 从推理的角度理解可以分为三个过程，VAE（变分自编码器）、U-NET（解码器）、CLIP Text Encoder（文本编码器）。理解了这三个模块，就入门了 Stabel Siffusion,下面我们逐一看一下这三个模块。

![sd 流程图](../images/05SD03.jpg)

VAE 模型主要起到图像的压缩和图像重建的作用，比如输入一个 H x W x C 的数据，同时比率为 f, 你会得到一个 H*f、W*f 高度、宽度的图像。

![sd2](../images/04SD02.png)

下面是 SD VAE 结构的整体 block 框图,主要是 GSC、Downsample、Upsample 三块。
![VAE block](../images/06SD04.jpg)
下面我们用调库的方式对 VAE 有一个直观的认识，当然想要深入的理解还是要仔细看一下源码。

In [ ]:
import cv2
import torch
import numpy as np
from diffusers import AutoencoderKL

VAE = AutoencoderKL.from_pretrained("/本地路径/stable-diffusion-v1-5", subfolder="vae")
VAE.to("cuda", dtype=torch.float16)

# 用 OpenCV 读取和调整图像大小
raw_image = cv2.imread("catwoman.png")
raw_image = cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB)
raw_image = cv2.resize(raw_image, (1024, 1024))

# 将图像数据转换为浮点数并归一化
image = raw_image.astype(np.float32) / 127.5 - 1.0

# 调整数组维度以匹配 PyTorch 的格式 (N, C, H, W)
image = image.transpose(2, 0, 1)
image = image[None, :, :, :]

# 转换为 PyTorch 张量
image = torch.from_numpy(image).to("cuda", dtype=torch.float16)

# 压缩图像为 Latent 特征并重建
with torch.inference_mode():
    # 使用 VAE 进行压缩和重建
    latent = VAE.encode(image).latent_dist.sample()
    rec_image = VAE.decode(latent).sample

    # 后处理
    rec_image = (rec_image / 2 + 0.5).clamp(0, 1)
    rec_image = rec_image.cpu().permute(0, 2, 3, 1).numpy()

    # 反归一化
    rec_image = (rec_image * 255).round().astype("uint8")
    rec_image = rec_image[0]

    # 保存重建后图像
    cv2.imwrite("reconstructed_catwoman.png", cv2.cvtColor(rec_image, cv2.COLOR_RGB2BGR))


U-Net 模块是 SD 模型的核心模块，能够预测噪声残差，并结合 Sampling method 对输入的特征矩阵进行重构，将随机高斯噪声转换成图片的 Latent Feature，其 block 示意图如下。具体解释参照正文。
![Unet](../images/06SD05.jpg)


clip 的部分就比较常见，这里不再赘述，参考前文，演示代码如下。

In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer

# 加载 CLIP Text Encoder 模型和 Tokenizer
# SD V1.5 模型权重百度云网盘：关注 Rocky 的公众号 WeThinkIn，后台回复：SDV1.5 模型，即可获得资源链接
text_encoder = CLIPTextModel.from_pretrained("/本地路径/stable-diffusion-v1-5", subfolder="text_encoder").to("cuda")
text_tokenizer = CLIPTokenizer.from_pretrained("/本地路径/stable-diffusion-v1-5", subfolder="tokenizer")

# 将输入 SD 模型的 prompt 进行 tokenize，得到对应的 token ids 特征
prompt = "1girl,beautiful"
text_token_ids = text_tokenizer(
    prompt,
    padding="max_length",
    max_length=text_tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt"
).input_ids

print("text_token_ids' shape:",text_token_ids.shape)
print("text_token_ids:",text_token_ids)

# 将 token ids 特征输入 CLIP Text Encoder 模型中输出 77x768 的 Text Embeddings 特征
text_embeddings = text_encoder(text_token_ids.to("cuda"))[0] # 由于 CLIP Text Encoder 模型输出的是一个元组，所以需要[0]对 77x768 的 Text Embeddings 特征进行提取
print("text_embeddings' shape:",text_embeddings.shape)
print(text_embeddings)


最终我们介绍一下 huggingface 的 diffusions 库。我们可以方便的调用现有的轮子，如下所示。

In [ ]:
import torch
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained(
  "Qwen/Qwen-Image", torch_dtype=torch.bfloat16, device_map="cuda"
)

prompt = """
cinematic film still of a cat sipping a margarita in a pool in Palm Springs, California
highly detailed, high budget hollywood movie, cinemascope, moody, epic, gorgeous, film grain
"""
pipeline(prompt).images[0]


我们可以自行选择自己喜欢的 VAE 组件来构成 pipeline。

In [ ]:
from diffusers import StableDiffusionXLPipeline, HeunDiscreteScheduler, AutoencoderKL
import torch

scheduler = HeunDiscreteScheduler.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="scheduler")
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16, use_safetensors=True)

pipeline = StableDiffusionXLPipeline.from_pretrained(
  "stabilityai/stable-diffusion-xl-base-1.0",
  scheduler=scheduler,
  vae=vae,
  torch_dtype=torch.float16,
  variant="fp16",
  use_safetensors=True
).to("cuda")
